In [1]:
%load_ext nb_black

<IPython.core.display.Javascript object>

In [2]:
# evaluate multinomial logistic regression model
import numpy as np
import pandas as pd
from sklearn.model_selection import *
from sklearn.metrics import confusion_matrix

# from sklearn.model_selection import cross_val_score, cross_validate
# from sklearn.model_selection import RepeatedStratifiedKFold
from sklearn.linear_model import LogisticRegression

import plotly.express as px
from IPython.display import display
import statsmodels.api as sm

C:\Users\Gianni\anaconda3\lib\site-packages\statsmodels\compat\pandas.py:61: FutureWarning: pandas.Int64Index is deprecated and will be removed from pandas in a future version. Use pandas.Index with the appropriate dtype instead.
  from pandas import Int64Index as NumericIndex


<IPython.core.display.Javascript object>

In [3]:
data = pd.read_csv("Data/data_clean.csv", index_col=[0])
display(data)

X = data.drop(["score", "elapse", "gender", "age"], axis=1)
Y = data["gender"]

X_train, X_test, Y_train, Y_test = train_test_split(
    X, Y, test_size=0.3, random_state=50
)

,score,Influence,Modesty,Daring,Confidence,Ruling,Responsibility,IntrovExtro,Success,Humble,...,Confidence in Success,MakeBelieve,Leadership Origin,Biography,Public Opinion,Capability,Extraordinary,elapse,gender,age
0,18,2,2,2,2,1,2,1,2,2,...,1,1,2,2,2,1,2,211,1,50
1,6,2,2,2,1,2,2,1,2,1,...,2,1,2,2,2,2,1,149,1,40
2,27,1,2,2,1,2,1,2,1,2,...,1,2,1,1,2,1,2,168,1,28
3,29,1,1,2,2,2,1,2,1,1,...,1,2,1,2,2,1,1,230,1,37
4,6,1,2,1,1,1,2,1,2,1,...,2,1,2,2,2,0,1,389,1,50
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
11238,1,2,2,2,1,1,2,1,2,1,...,2,1,2,2,2,2,1,243,1,24
11239,10,2,2,1,1,1,2,1,1,1,...,2,1,2,1,2,2,1,197,1,39
11240,6,1,2,2,1,1,2,1,2,1,...,2,1,2,2,2,2,1,447,2,33
11241,12,2,2,1,1,1,1,1,1,1,...,1,2,2,2,2,2,1,167,1,24


<IPython.core.display.Javascript object>

In [10]:
# Building the Multinomial Logistic Model with sklearn
model = LogisticRegression(
    multi_class="multinomial", solver="sag", fit_intercept=False, max_iter=100
)

cv = RepeatedStratifiedKFold(n_splits=10, n_repeats=4, random_state=1)
n_scores = cross_validate(
    model, X_train, Y_train, scoring="accuracy", cv=cv, n_jobs=-1, return_estimator=True
)

print(
    "Mean Accuracy: %.3f (+/- %.3f)"
    % (np.mean(n_scores["test_score"]), np.std(n_scores["test_score"]))
)

# print(np.argmin(n_scores["test_score"]))

Mean Accuracy: 0.639 (+/- 0.017)


<IPython.core.display.Javascript object>

In [11]:
# for model in n_scores["estimator"]:
#     print(model.coef_)

# n_scores["estimator"].coef_

model.fit(X_train, Y_train)

# Create dataframe for coefficients
model_coef = pd.DataFrame(model.coef_.T)
model_coef.insert(0, "Questions", X.columns)
model_coef.columns = ["Questions", "Male", "Female", "Other"]
Y_pred = model.predict(X_test)

# Create dataframe to assess predictions
model_eval = pd.DataFrame({"Actual": Y_test, "Predicted": Y_pred})
display(model_eval)
display(model_coef)

,Actual,Predicted
591,1,1
2460,2,1
3615,2,2
2395,1,1
6474,1,1
...,...,...
3270,1,2
517,1,2
4199,2,2
2111,2,2


,Questions,Male,Female,Other
0,Influence,-0.153235,-0.197327,0.350562
1,Modesty,0.027687,0.238573,-0.266260
2,Daring,0.056730,0.052533,-0.109264
3,Confidence,0.103753,-0.056097,-0.047656
4,Ruling,0.491000,-0.009354,-0.481646
5,Responsibility,-0.066401,0.080226,-0.013826
6,IntrovExtro,0.032245,0.385599,-0.417844
7,Success,0.101538,-0.012117,-0.089421
8,Humble,0.297089,0.193029,-0.490118
9,Leadership,0.094736,0.140724,-0.235461


<IPython.core.display.Javascript object>

In [6]:
# Using Statsmodels.api
# logit_model = sm.MNLogit(Y, X)
# result = logit_model.fit()
# print(result.summary2())

<IPython.core.display.Javascript object>

In [12]:
conmat = confusion_matrix(Y_test, Y_pred)
display(conmat)
fig = px.imshow(
    np.round((conmat / conmat.sum()) * 100, 3),
    labels=dict(x="Predicted Values", y="True Values", color="Percentage Classified"),
    x=["Male", "Female", "Other"],
    y=["Male", "Female", "Other"],
)
fig.update_xaxes(side="top")
fig.show()

array([[1445,  482,    1],
       [ 727,  668,    0],
       [   9,    4,    0]], dtype=int64)

<IPython.core.display.Javascript object>

### Multinomial Log Reg with Age Group

In [16]:
data = pd.read_csv("Data/data_AgeGroup.csv", index_col=[0])
display(data)

X = data.drop(["score", "elapse", "gender", "age", "Age_Group"], axis=1)
Y = data["Age_Group"]

X_train, X_test, Y_train, Y_test = train_test_split(
    X, Y, test_size=0.3, random_state=50
)

,score,Influence,Modesty,Daring,Confidence,Ruling,Responsibility,IntrovExtro,Success,Humble,...,MakeBelieve,Leadership Origin,Biography,Public Opinion,Capability,Extraordinary,elapse,gender,age,Age_Group
0,18,2,2,2,2,1,2,1,2,2,...,1,2,2,2,1,2,211,1,50,2.0
1,6,2,2,2,1,2,2,1,2,1,...,1,2,2,2,2,1,149,1,40,2.0
2,27,1,2,2,1,2,1,2,1,2,...,2,1,1,2,1,2,168,1,28,2.0
3,29,1,1,2,2,2,1,2,1,1,...,2,1,2,2,1,1,230,1,37,2.0
4,6,1,2,1,1,1,2,1,2,1,...,1,2,2,2,0,1,389,1,50,2.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
11238,1,2,2,2,1,1,2,1,2,1,...,1,2,2,2,2,1,243,1,24,1.0
11239,10,2,2,1,1,1,2,1,1,1,...,1,2,1,2,2,1,197,1,39,2.0
11240,6,1,2,2,1,1,2,1,2,1,...,1,2,2,2,2,1,447,2,33,2.0
11241,12,2,2,1,1,1,1,1,1,1,...,2,2,2,2,2,1,167,1,24,1.0


<IPython.core.display.Javascript object>

In [17]:
# Building the Multinomial Logistic Model with sklearn
model = LogisticRegression(
    multi_class="multinomial", solver="sag", fit_intercept=False, max_iter=100
)

cv = RepeatedStratifiedKFold(n_splits=10, n_repeats=4, random_state=1)
n_scores = cross_validate(
    model, X_train, Y_train, scoring="accuracy", cv=cv, n_jobs=-1, return_estimator=True
)

print(
    "Mean Accuracy: %.3f (+/- %.3f)"
    % (np.mean(n_scores["test_score"]), np.std(n_scores["test_score"]))
)

Mean Accuracy: 0.669 (+/- 0.013)


<IPython.core.display.Javascript object>

In [18]:
model.fit(X_train, Y_train)

# Create dataframe for coefficients
model_coef = pd.DataFrame(model.coef_.T)
model_coef.insert(0, "Questions", X.columns)
model_coef.columns = ["Questions", "Male", "Female", "Other"]
Y_pred = model.predict(X_test)

# Create dataframe to assess predictions
model_eval = pd.DataFrame({"Actual": Y_test, "Predicted": Y_pred})
display(model_eval)
display(model_coef)

,Actual,Predicted
7193,2.0,2.0
6767,1.0,2.0
6315,2.0,2.0
4855,2.0,2.0
4796,2.0,2.0
...,...,...
8811,1.0,2.0
9294,3.0,2.0
10551,2.0,2.0
2294,2.0,2.0


,Questions,Male,Female,Other
0,Influence,0.226722,0.038218,-0.264940
1,Modesty,-0.142314,-0.010868,0.153183
2,Daring,-0.221389,-0.210236,0.431625
3,Confidence,0.104234,-0.124376,0.020142
4,Ruling,0.153934,-0.060860,-0.093074
5,Responsibility,-0.302526,-0.028033,0.330560
6,IntrovExtro,0.088241,0.150829,-0.239071
7,Success,-0.497577,-0.138751,0.636328
8,Humble,0.192379,0.157520,-0.349899
9,Leadership,0.172179,0.159044,-0.331222


<IPython.core.display.Javascript object>

In [19]:
conmat = confusion_matrix(Y_test, Y_pred)
display(conmat)
fig = px.imshow(
    np.round((conmat / conmat.sum()) * 100, 3),
    labels=dict(x="Predicted Values", y="True Values", color="Percentage Classified"),
    x=["Male", "Female", "Other"],
    y=["Male", "Female", "Other"],
)
fig.update_xaxes(side="top")
fig.show()

array([[ 269,  770,    0],
       [ 195, 1889,    0],
       [   4,   88,    0]], dtype=int64)

<IPython.core.display.Javascript object>